<a href="https://colab.research.google.com/github/lolonao/code-playground/blob/main/%E5%A4%96%E9%83%A8%E3%83%87%E3%83%BC%E3%82%BF_%E3%83%AD%E3%83%BC%E3%82%AB%E3%83%AB_%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%80%81%E3%83%89%E3%83%A9%E3%82%A4%E3%83%96%E3%80%81%E3%82%B9%E3%83%97%E3%83%AC%E3%83%83%E3%83%89%E3%82%B7%E3%83%BC%E3%83%88%E3%80%81Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このノートブックには、外部ソースからのデータの読み込みと保存に関するレシピが記載されています。

# ローカル ファイル システム

## ローカル ファイル システムからファイルをアップロードする

<code>files.upload</code> は、アップロードされたファイルの辞書を返します。
この辞書ではファイル名がキーとなります。値はアップロードされたデータです。

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## ローカル ファイル システムにファイルをダウンロードする

<code>files.download</code> は、ブラウザによるファイルのダウンロードをローカルのパソコンに呼び出します。


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google ドライブ

次のような方法でドライブ内のファイルにアクセスできます。
- ランタイムの仮想マシンで Google ドライブをマウントする
- API のラッパーを使用する（<a href="https://pythonhosted.org/PyDrive/">PyDrive</a> など）
- <a href="https://developers.google.com/drive/v3/web/about-sdk">ネイティブ REST API</a> を使用する



それぞれの例を以下に示します。

## Google ドライブをローカルにマウントする

以下の例では、認証コードを使用してランタイム上に Google ドライブをマウントする方法と、そのドライブでファイルを読み書きする方法を示します。実行後は、<a href="https://drive.google.com/">https://drive.google.com/</a> で新しいファイル（<code>foo.txt</code>）を確認できます。

この例では、ファイルの読み取り、書き込み、移動のみに対応しています。共有設定や他のメタデータをプログラムから変更するには、下記の他の方法を使用してください。

<strong>注:</strong> ファイル ブラウザで &#91;ドライブをマウント&#93; ボタンを使用する場合、現在のユーザーのみに編集されたノートブックについては認証コードは必要ありません。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive

下の例では、PyDrive を使用した認証、ファイルのアップロードとダウンロードを示しています。その他の例については、<a href="https://pythonhosted.org/PyDrive/">PyDrive のドキュメント</a>をご覧ください。

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

認証を行い、PyDrive クライアントを作成します。


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

テキスト ファイルを作成してアップロードします。


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


ID を指定してファイルを読み込み、その内容を出力します。


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive REST API

Drive API を使用するには、最初に認証を行ってから、API クライアントを作成する必要があります。


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

このクライアントでは、<a href="https://developers.google.com/drive/v3/reference/">Google Drive API リファレンス</a>に記載されているどの関数でも使用できます。例をご覧ください。


### Python からのデータを含む新しいドライブ ファイルを作成する

まず、アップロードするローカル ファイルを作成します。

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

<a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a> メソッドを使用してファイルをアップロードします。ファイルのアップロードについて詳しくは、<a href="https://developers.google.com/drive/v3/web/manage-uploads">デベロッパー ドキュメント</a>をご覧ください。

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


上記のセルを実行すると、「Sample file」という名前の新しいファイルが <a href="https://drive.google.com/">https://drive.google.com/</a> に表示されます。

### ドライブ ファイルから Python にデータをダウンロードする

上記でアップロードしたファイルをダウンロードします。

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


別のファイルをダウンロードするには、上記の <code>file&#95;id</code> にそのファイルの ID を設定します（例: 1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz）。

# Google スプレッドシート

以下の例では、Google スプレッドシートとのやり取りにオープンソースの <a href="https://github.com/burnash/gspread"><code>gspread</code></a> ライブラリを使用しています。

ライブラリをインポートし、認証を行い、スプレッドシートへのインターフェースを作成します。

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

以下に示すのは <code>gspread</code> の例の一部分です。その他の例については、<a href="https://github.com/burnash/gspread#more-examples"><code>gspread</code> に関する GitHub ページ</a>をご覧ください。

## Python からのデータを含む新しいシートを作成する

In [ ]:
sh = gc.create('My cool spreadsheet')

上記のセルを実行すると、「My cool spreadsheet」という名前の新しいスプレッドシートが <a href="https://sheets.google.com/">https://sheets.google.com</a> に表示されます。

新しいシートを開いて、何かしらのデータを追加します。

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

## シートから Python にデータを Pandas DataFrame としてダウンロードする

上記で挿入したランダムなデータを再び読み取り、結果を <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">Pandas DataFrame</a> に変換します。

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# Google Cloud Storage（GCS）

GCS で Colaboratory を使用するには、<a href="https://cloud.google.com/storage/docs/projects">Google Cloud プロジェクト</a>を作成するか、既存のプロジェクトを使用する必要があります。

以下でプロジェクト ID を指定します。

In [ ]:
project_id = 'Your_project_ID_here'

GCS のファイルは<a href="https://cloud.google.com/storage/docs/buckets">バケット</a>に格納されます。

バケットにはグローバルに一意の名前を付ける必要があるため、ここで名前を生成します。

In [ ]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

GCS にアクセスするには、認証を行う必要があります。

In [ ]:
from google.colab import auth
auth.authenticate_user()

GCS には <code>gsutil</code> コマンドライン ユーティリティかネイティブ Python API を使用してアクセスできます。

## `gsutil`

まず、<code>gcloud</code> を使用して、上記で指定したプロジェクトを使用するように <code>gsutil</code> を設定します。

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


アップロードするローカル ファイルを作成します。

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

ファイルのアップロード先となるバケットを作成します（<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">ドキュメント</a>）。

In [ ]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


ファイルを新しいバケットにコピーします（<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">ドキュメント</a>）。

In [ ]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


新しくコピーしたファイルの内容をダンプして、問題がないことを確認します（<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">ドキュメント</a>）。


In [ ]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [ ]:
#@markdown アップロードが終了すると、プロジェクトの Cloud コンソール ストレージ ブラウザにデータが表示されます。
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


最後に、上記の例でアップロードしたファイルをダウンロードします。<code>gsutil cp</code> コマンドで順序を逆にするだけなので簡単です。

In [ ]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# 結果を出力して、転送が機能していることを確認します。
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## Python API

以下のスニペットは API のその他の用法を示した<a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">より大きなコード例</a>に基づいています。

まず、サービス クライアントを作成します。

In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

アップロードするローカル ファイルを作成します。

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

上記で指定したプロジェクト内にバケットを作成します。

In [ ]:
# 上記の gsutil の例から、グローバルに一意な別のバケット名を使用します。
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


新しく作成したバケットにファイルをアップロードします。

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [ ]:
#@markdown アップロードが終了すると、プロジェクトの Cloud コンソール ストレージ ブラウザにデータが表示されます。
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


アップロードしたファイルをダウンロードします。

In [ ]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


ダウンロードしたファイルを確認します。


In [ ]:
!cat /tmp/downloaded_from_gcs.txt

my sample file